# Predicting number of corners per room

In the HouseDiffusion paper, the number of corners per room are sampled conditioned only on the room_type. They did it by counting how often each number_of_corners was used per room type, and sampling each room independently based on the room type.

As I already had a notebook for predicting the room type from the zoning type, I tried using the same method for predicting number of corners. However the accuracy turned out to be quite a bit lower for predicting number of corners compared to predicting room type.


This notebook is based on the `node_classification_room_type.ipynb` one. The `node_classification_room_type.ipynb` has to be run first, as this notebook uses the outputted room_type as feature.

In [1]:
import os
import torch
import torch_geometric as pyg
import numpy as np
import networkx as nx
from utils import load_pickle

import constants

In [2]:
from data_preprocessing.process_graphs import simplify_room_polygon
from shapely import geometry

In [3]:
def add_zoning_attribute(graph: nx.Graph):
    room_type = nx.get_node_attributes(graph, 'room_type')

    room_names = {node: constants.ROOM_NAMES[value] for node, value in room_type.items()}

    inv_room_mapping = {val: key for key, val in constants.ROOM_MAPPING.items()}

    room_names = {node: inv_room_mapping[value] for node, value in room_names.items()}

    zoning = {key: constants.ZONING_MAPPING[value] for key, value in room_names.items()}

    zoning_index = {key: constants.ZONING_NAMES.index(value) for key, value in zoning.items()}

    nx.set_node_attributes(graph, zoning_index, 'zoning_type')



In [4]:
# MAX_CORNERS = 128

In [5]:
from sklearn.model_selection import train_test_split

def one_hot_encode(value, num_classes):
    return torch.eye(num_classes)[value]


NUM_ROOM_TYPES = 9
NUM_ZONING_TYPES = 4

# def one_hot_encode_types(graph: nx.Graph):
#     for node in graph.nodes:
#         graph.nodes[node]['room_type'] = one_hot_encode(graph.nodes[node]['room_type'], NUM_ROOM_TYPES)
#         graph.nodes[node]['zoning_type'] = one_hot_encode(graph.nodes[node]['zoning_type'], NUM_ZONING_TYPES)

CONNECTIVITIES = ["door", "entrance", "passage"]

class GraphZoningRoomTypeDataset(torch.utils.data.Dataset):
    """
    Graph Dataset. Collects NetworkX graph from a pre-defined folder and
    transforms them to Pytorch Geometric (pyg.data.Data()) instances.
    """
    def __init__(self, path, split="train"):
        # self.graph_in_path = os.path.join(path, 'graph_in')
        self.graph_out_path = os.path.join(path, 'graph_out')

        # include graph transformations if you like
        # self.graph_transform = graph_transform

        all_files = os.listdir(self.graph_out_path)

        self.train_files, self.val_files = train_test_split(all_files, test_size=0.05, random_state=42)

        if split == "train":
            self.files = self.train_files
        elif split == "val":
            self.files = self.val_files
        else:
            raise ValueError(f"Invalid split: {split}")
        
        self.cache = {}

    def __getitem__(self, index):

        if index in self.cache:
            graph_nx = self.cache[index]
        else:
            file_name = self.files[index]

            # get access graph (name is index)
            graph_nx = load_pickle(os.path.join(self.graph_out_path, file_name))

            add_zoning_attribute(graph_nx)

            for edge in graph_nx.edges:
                graph_nx.edges[edge]['connectivity'] = CONNECTIVITIES.index(graph_nx.edges[edge]['connectivity'])

            for node in graph_nx.nodes:
                polygon = geometry.Polygon(graph_nx.nodes[node]['geometry'])

                polygon = simplify_room_polygon(polygon)

                graph_nx.nodes[node]['n_corners'] = float(len(polygon.exterior.coords))

                del graph_nx.nodes[node]['geometry']

                # graph_nx.nodes[node]["polygon"] = np.array(polygon.exterior.coords)

            # Remove attributes geometry, centroid
            for node in graph_nx.nodes:
                # graph_nx.nodes[node].pop('geometry', None)
                graph_nx.nodes[node].pop('centroid', None)

            # graph_nx.graph[]

            self.cache[index] = graph_nx

        # transform networkx graph to pytorch geometric graph
        graph_pyg = pyg.utils.from_networkx(graph_nx)

        # transform graph if you like
        graph_pyg = self.graph_transform(graph_pyg)

        return graph_pyg

    @staticmethod
    def graph_transform(graph_pyg):
        graph_pyg["room_type"] = one_hot_encode(graph_pyg["room_type"], NUM_ROOM_TYPES)
        graph_pyg["zoning_type"] = one_hot_encode(graph_pyg["zoning_type"], NUM_ZONING_TYPES)

        graph_pyg["connectivity"] = one_hot_encode(graph_pyg["connectivity"], len(CONNECTIVITIES))

        # graph_pyg["n_corners"] = one_hot_encode(graph_pyg["n_corners"], MAX_CORNERS)

        graph_pyg["n_corners"] = torch.unsqueeze(graph_pyg["n_corners"], dim=-1)

        return graph_pyg

    def __len__(self):
        return len(self.files)


In [6]:

class GraphZoningTypeTestSet(torch.utils.data.Dataset):
    """
    Graph Dataset. Collects NetworkX graph from a pre-defined folder and
    transforms them to Pytorch Geometric (pyg.data.Data()) instances.
    """
    def __init__(self, path, graph_name="graph_pred"):
        self.graph_path = os.path.join(path, graph_name)

        self.files = os.listdir(self.graph_path)

    def __getitem__(self, index):

        file_name = self.files[index]

        # get access graph (name is index)
        graph_nx = load_pickle(os.path.join(self.graph_path, file_name))

        for edge in graph_nx.edges:
            graph_nx.edges[edge]['connectivity'] = CONNECTIVITIES.index(graph_nx.edges[edge]['connectivity'])

        # transform networkx graph to pytorch geometric graph
        graph_pyg = pyg.utils.from_networkx(graph_nx)

        # transform graph if you like
        graph_pyg = self.graph_transform(graph_pyg)

        graph_pyg["file_name"] = file_name

        return graph_pyg

    @staticmethod
    def graph_transform(graph_pyg):
        graph_pyg["room_type"] = one_hot_encode(graph_pyg["room_type"], NUM_ROOM_TYPES)

        graph_pyg["zoning_type"] = one_hot_encode(graph_pyg["zoning_type"], NUM_ZONING_TYPES)

        graph_pyg["connectivity"] = one_hot_encode(graph_pyg["connectivity"], len(CONNECTIVITIES))

        return graph_pyg

    def __len__(self):
        return len(self.files)


In [7]:
path = "/path/to/modified-swiss-dwellings-train/"

# graph_nx = load_pickle(os.path.join(graph_path, f'{43}.pickle'))

In [8]:
ds_train = GraphZoningRoomTypeDataset(path, split="train")

ds_val = GraphZoningRoomTypeDataset(path, split="val")

In [9]:
for index in range(len(ds_val)):
    ds_val[index]["n_corners"]

In [10]:
# import shapely.geometry as sg



# print(len(ds_train[0]["polygon"][14]))

# display(sg.Polygon(ds_train[0]["polygon"][14]).simplify(tolerance=0.1, preserve_topology=True))


# polygon = sg.Polygon(ds_train[0]["polygon"][14])
# prev_len = 100000

# while len(polygon.simplify(0.1).exterior.coords) < prev_len:
#     polygon = polygon.simplify(0.1)
#     prev_len = len(polygon.exterior.coords)

# prev_len
# # print(len(.simplify(.1).exterior.coords))

In [16]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

from torch import nn

class GATModel(torch.nn.Module):
    def __init__(self, num_features=NUM_ROOM_TYPES, num_edge_features=len(CONNECTIVITIES), hidden_size=32, target_size=1, num_hidden_layers=2):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_features = num_features
        self.num_edge_features = num_edge_features
        self.target_size = target_size

        self.convs = nn.ModuleList([GATConv(self.num_features if i == 0 else self.hidden_size, self.hidden_size, edge_dim=self.num_edge_features) for i in range(num_hidden_layers)])

        if num_hidden_layers == 0:
            self.hidden_linear = nn.Linear(self.num_features, self.hidden_size)
        else:
            self.hidden_linear = nn.Linear(self.hidden_size + num_features, self.hidden_size)

        self.linear = nn.Linear(self.hidden_size, self.target_size)

    def forward(self, data):
        x, edge_index, edge_attr = data.room_type, data.edge_index, data.connectivity

        for conv in self.convs:
            x = conv(x, edge_index, edge_attr=edge_attr) # adding edge features here!
            x = F.relu(x)
            x = F.dropout(x, training=self.training)

        # x = self.convs[-1](x, edge_index, edge_attr=edge_attr) # edge features here as well
        
        if len(self.convs) > 0:
            x = torch.cat([x, data.room_type], dim=-1)

        x = self.hidden_linear(x)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.linear(x)

        return F.relu(x) 

In [17]:
# model

In [18]:
from torch_geometric.loader import DataLoader

from collections import Counter

# loss_func = torch.nn.CrossEntropyLoss()
# loss_func = torch.nn.MSELoss()
loss_func = torch.nn.L1Loss()


def evaluate_model(model, data_val, device="cpu"):
    model.eval()

    total_loss = 0
    accuracies = []

    absolute_distances = []

    model = model.to(device)

    with torch.no_grad():

        # Batch size should be 1, because want to evaluate each graph separately
        for data in DataLoader(data_val, batch_size=1):
            data = data.to(device)
            out = model(data)

            pred = model(data.to(device)).squeeze()
            
            # pred = torch.argmax(pred, dim=-1)

            gt = data.n_corners.squeeze()
            
            # gt = torch.argmax(data.n_corners, dim=-1).to(device)
            
            
            # Tensor of shape (batch_size, 1)
            acc = (torch.abs(gt - pred) < 1).sum(dim=-1) / pred.shape[-1]
            acc = acc.item()
            # acc = 0

            absolute_distance = torch.abs(gt - pred).mean(dim=-1, dtype=torch.float32)
            
#             print(pred.shape, gt.shape)
            
#             print(absolute_distance.shape)

            # pred_counter = Counter(pred.cpu().numpy())
            # gt_counter = Counter(gt.cpu().numpy())

            # iou = sum((pred_counter & gt_counter).values()) / sum((pred_counter | gt_counter).values())
            # ious.append(iou)

            accuracies.append(acc)

            absolute_distances.append(absolute_distance.item())
            
            val_loss = loss_func(out, data.n_corners)

            total_loss += val_loss.item()
    
    model.train()
    
    return total_loss, np.mean(accuracies), np.mean(absolute_distances), np.std(absolute_distances)


def train(model, data_train, data_val, batch_size, learning_rate, n_epochs=1, device="cpu", save_loss_interval=1, print_interval=1):

    NUM_TRAIN = len(data_train)
    NUM_VAL = len(data_val)

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)

    # test_data = data_test[0]

    model = model.to(device)

    for epoch in range(n_epochs):
        epoch_loss = 0
        model.train()
        
        for data in loader:
            
            data = data.to(device)

            optimizer.zero_grad()
            
            out = model(data)
            
            loss = loss_func(out, data.n_corners)
            
            epoch_loss += loss.item() 
            loss.backward()
            optimizer.step()

        if epoch % save_loss_interval == 0:
            val_loss, val_acc, mean_absolute_error, std_absolute_error = evaluate_model(model, data_val, device="cpu")

            val_loss /= NUM_VAL

            model = model.to(device)

            train_loss = epoch_loss / NUM_TRAIN
          
            if epoch % print_interval == 0:
                print(f"Epoch: {epoch} Train loss: {train_loss:.3e} Val loss: {val_loss:.3e} Val acc: {val_acc:.3f} Val mean absolute error: {mean_absolute_error:.3f} Val std absolute error: {std_absolute_error:.3f}")
          
            yield {
                "epoch": epoch,
                "train_loss": train_loss,
                "val_loss": val_loss,
                "val_acc": val_acc,
                "val_mean_absolute_error": mean_absolute_error,
                "val_std_absolute_error": std_absolute_error
            }

In [19]:
import copy

def train_with_early_stopping(model, ds_train, ds_val, batch_size=32, learning_rate=0.001, n_epochs=100, device="cuda", tolerance=5):

    losses = []

    best_model = (None, 1e5, None)

    for loss in train(model, ds_train, ds_val, batch_size=batch_size, learning_rate=learning_rate, n_epochs=n_epochs, device=device):
        losses.append(loss)

        if loss["val_loss"] < best_model[1]:
            best_model = (copy.deepcopy(model), loss["val_loss"], loss)

            print(f"New best model with val loss: {loss['val_loss']:.3e}")
        
        if loss["epoch"] - best_model[2]["epoch"] > tolerance:
            print(f"Stopping early at epoch {loss['epoch']}")
            break

    return {
        "losses": losses,
        "best_model": best_model[0],
        "best_model_val_loss": best_model[1],
        "best_model_loss_dict": best_model[2],
        "last_model": model,
    }

# train_with_early_stopping(model, ds_train, ds_val)

In [24]:
for num_hidden_layers in [3]:
    model = GATModel(num_hidden_layers=num_hidden_layers)

    print(f"Training model with {num_hidden_layers} hidden layers")

    result = train_with_early_stopping(model, ds_train, ds_val, batch_size=32, learning_rate=0.001, n_epochs=100, device="cuda", tolerance=5)

    torch.save(result, f"n_corners_classifier_mse_early_stopping_results_{num_hidden_layers}.pt")

Training model with 3 hidden layers
Epoch: 0 Train loss: 1.231e-01 Val loss: 1.796e+00 Val acc: 0.535 Val mean absolute error: 1.796 Val std absolute error: 0.745
New best model with val loss: 1.796e+00
Epoch: 1 Train loss: 7.445e-02 Val loss: 1.746e+00 Val acc: 0.511 Val mean absolute error: 1.746 Val std absolute error: 0.702
New best model with val loss: 1.746e+00
Epoch: 2 Train loss: 7.025e-02 Val loss: 1.712e+00 Val acc: 0.520 Val mean absolute error: 1.712 Val std absolute error: 0.693
New best model with val loss: 1.712e+00
Epoch: 3 Train loss: 6.742e-02 Val loss: 1.684e+00 Val acc: 0.539 Val mean absolute error: 1.684 Val std absolute error: 0.706
New best model with val loss: 1.684e+00
Epoch: 4 Train loss: 6.507e-02 Val loss: 1.673e+00 Val acc: 0.543 Val mean absolute error: 1.673 Val std absolute error: 0.720
New best model with val loss: 1.673e+00
Epoch: 5 Train loss: 6.384e-02 Val loss: 1.671e+00 Val acc: 0.539 Val mean absolute error: 1.671 Val std absolute error: 0.687
Ne

## Attempt to train longer

Train longer with increasing batch size, gets slightly better results. However the accuracy, and std absolute error are still similar to above that trained only once.

It seems predicting the number of corners per room is a lot harder than predicting room_type. This would also make sense, as the number of corners would depend a lot more on the structural walls compared to the room_type, but structural walls isn't given as an input.

In [35]:
# model = GATModel(num_hidden_layers=1)

for batch_size in [16, 32, 64, 128, 256, 512]:
    result = train_with_early_stopping(model, ds_train, ds_val, batch_size=batch_size, learning_rate=0.0005, n_epochs=400, device="cuda", tolerance=10)


Epoch: 0 Train loss: 9.680e-02 Val loss: 1.542e+00 Val acc: 0.509 Val mean absolute error: 1.542 Val std absolute error: 0.720
New best model with val loss: 1.542e+00
Epoch: 1 Train loss: 9.715e-02 Val loss: 1.543e+00 Val acc: 0.568 Val mean absolute error: 1.543 Val std absolute error: 0.723
Epoch: 2 Train loss: 9.662e-02 Val loss: 1.542e+00 Val acc: 0.565 Val mean absolute error: 1.542 Val std absolute error: 0.719
New best model with val loss: 1.542e+00
Epoch: 3 Train loss: 9.691e-02 Val loss: 1.544e+00 Val acc: 0.566 Val mean absolute error: 1.544 Val std absolute error: 0.722
Epoch: 4 Train loss: 9.722e-02 Val loss: 1.543e+00 Val acc: 0.565 Val mean absolute error: 1.543 Val std absolute error: 0.722
Epoch: 5 Train loss: 9.694e-02 Val loss: 1.540e+00 Val acc: 0.564 Val mean absolute error: 1.540 Val std absolute error: 0.717
New best model with val loss: 1.540e+00
Epoch: 6 Train loss: 9.698e-02 Val loss: 1.542e+00 Val acc: 0.566 Val mean absolute error: 1.542 Val std absolute erro

In [39]:
best_model_dict = result["best_model_loss_dict"]

model = result["best_model"]

In [41]:
torch.save(result, "n_corners_classifier_mse_early_stopping_results:best.pt")

In [25]:
val_loss_dicts = []

for num_hidden_layers in [0, 1, 2, 3, 4, 8, 16]:
    result_i = torch.load( f"n_corners_classifier_mse_early_stopping_results_{num_hidden_layers}.pt", map_location="cpu")

    print(result_i["best_model_val_loss"], result_i["best_model_loss_dict"]["val_acc"])

    val_loss_dicts.append({
        "num_hidden_layers": num_hidden_layers,
        "val_loss": result_i["best_model_val_loss"],
        # "max_epoch": result_i["losses"][-1]["epoch"],
        "val_acc": result_i["best_model_loss_dict"]["val_acc"],
    })

1.6633778779130233 0.5360361379370735
1.5562544480750435 0.5636744407945843
1.5851912213284434 0.5406600720645708
1.594444824748062 0.5266760762727432
1.6629817731072458 0.5364675197090829
1.6646894507430958 0.5364675197090829
1.663627598559457 0.5360361379370735


In [42]:
# result_i = torch.load( f"n_corners_classifier_mse_early_stopping_results_{1}.pt", map_location="cpu")

result_i = torch.load("n_corners_classifier_mse_early_stopping_results:best.pt", map_location="cpu")

result_i.keys()

dict_keys(['losses', 'best_model', 'best_model_val_loss', 'best_model_loss_dict', 'last_model'])

In [43]:
result_i["best_model"]

GATModel(
  (convs): ModuleList(
    (0): GATConv(9, 32, heads=1)
  )
  (hidden_linear): Linear(in_features=41, out_features=32, bias=True)
  (linear): Linear(in_features=32, out_features=1, bias=True)
)

In [44]:
# import pandas as pd

# pd.DataFrame(val_loss_dicts).plot.bar(x="num_hidden_layers", y="val_acc")

In [93]:
print(pd.DataFrame(val_loss_dicts)[["num_hidden_layers", "val_loss", "val_acc"]].style.hide(axis="index").format(lambda x: f"{x:.2f}", ["val_loss", "val_acc"]).to_latex().replace("_", " "))



\begin{tabular}{rrr}
num hidden layers & val loss & val acc \\
2 & 0.35 & 0.87 \\
3 & 0.30 & 0.90 \\
4 & 0.35 & 0.87 \\
8 & 0.41 & 0.83 \\
16 & 0.58 & 0.74 \\
\end{tabular}



In [20]:
# num_hidden_layers = 3
# results_dict = torch.load(f"room_type_classifier_early_stopping_results_{num_hidden_layers}.pt")

# model = results_dict["best_model"]

In [45]:
model.eval()

GATModel(
  (convs): ModuleList(
    (0): GATConv(9, 32, heads=1)
  )
  (hidden_linear): Linear(in_features=41, out_features=32, bias=True)
  (linear): Linear(in_features=32, out_features=1, bias=True)
)

In [49]:
ds_test = GraphZoningTypeTestSet('/path/to/modified-swiss-dwellings-v1-test/', "graph_pred")

ds_test[0]

Data(edge_index=[2, 80], zoning_type=[38, 4], room_type=[38, 9], connectivity=[80, 3], num_nodes=38, file_name='4167.pickle')

In [31]:
ds_test

In [ ]:
pred_graph_path = ds_test.graph_path.replace("graph_pred", "graph_pred_n_corners")

os.makedirs(pred_graph_path, exist_ok=True)

pred_graph_path

In [51]:
import pickle

def save_pickle(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f)

In [52]:
def inference(model, data_test, device="cpu"):
    model.eval()

    model = model.to(device)

    with torch.no_grad():
        for data in DataLoader(data_test, batch_size=1):
            data = data.to(device)
            pred = model(data)

            pred = pred.cpu().numpy()

            file_name = data.file_name

            assert len(file_name) == 1

            yield file_name[0], pred

inv_room_mapping = {val: key for key, val in constants.ROOM_MAPPING.items()}

for file_name, pred in inference(model, ds_test, device="cpu"):
    graph_nx = load_pickle(os.path.join(ds_test.graph_path, file_name))

    for node in graph_nx.nodes:
        graph_nx.nodes[node]['n_corners'] = pred[node]

    # print(graph_nx.nodes(data=True))

    print(file_name)
    
    save_pickle(graph_nx, os.path.join(pred_graph_path, file_name))

4167.pickle
4168.pickle
4169.pickle
4170.pickle
4171.pickle
4172.pickle
4173.pickle
4174.pickle
4175.pickle
4176.pickle
4177.pickle
4178.pickle
4179.pickle
4180.pickle
4181.pickle
4182.pickle
4183.pickle
4184.pickle
4185.pickle
4186.pickle
4187.pickle
4188.pickle
4189.pickle
4190.pickle
4191.pickle
4192.pickle
4193.pickle
4194.pickle
4195.pickle
4196.pickle
4197.pickle
4198.pickle
4199.pickle
4200.pickle
4201.pickle
4202.pickle
4203.pickle
4204.pickle
4205.pickle
4206.pickle
4207.pickle
4208.pickle
4209.pickle
4210.pickle
4211.pickle
4212.pickle
4213.pickle
4214.pickle
4215.pickle
4216.pickle
4217.pickle
4218.pickle
4219.pickle
4220.pickle
4221.pickle
4222.pickle
4223.pickle
4224.pickle
4225.pickle
4226.pickle
4227.pickle
4228.pickle
4229.pickle
4230.pickle
4231.pickle
4232.pickle
4233.pickle
4234.pickle
4235.pickle
4236.pickle
4237.pickle
4238.pickle
4239.pickle
4240.pickle
4241.pickle
4242.pickle
4243.pickle
4244.pickle
4245.pickle
4246.pickle
4247.pickle
4248.pickle
4249.pickle
4250

In [ ]:
ds_test.graph_path

In [255]:
model.to("cuda")

GATModel(
  (convs): ModuleList(
    (0): GATConv(4, 32, heads=1)
    (1): GATConv(32, 32, heads=1)
  )
  (linear): Linear(in_features=32, out_features=9, bias=True)
)